In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
base_folder = "/content/drive/MyDrive/housing_app_fall25"
%cd "{base_folder}"


/content/drive/MyDrive/housing_app_fall25


In [9]:
from pathlib import Path
import os
import sqlite3
import csv

# 0) Set your repo base folder (you already have this in your notebook usually)
# Example:
base_folder = "/content/drive/MyDrive/housing_app_fall25"
# If base_folder is already set above, keep it.
BASE = Path(base_folder)

DATA_CSV = BASE / "data" / "WA_Fn-UseC_-Telco-Customer-Churn.csv"
assert DATA_CSV.exists(), f"CSV not found at: {DATA_CSV}"

# ---- 1) NO-PANDAS CSV parsing (REQUIREMENT) ----
def load_telco_rows():
    print("[1] Loading Telco CSV using csv.DictReader (NO pandas)…")
    rows = []
    with open(DATA_CSV, "r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for r in reader:
            rows.append({k: (v.strip() if isinstance(v, str) else v) for k, v in r.items()})
    print(f"[2] Loaded {len(rows)} rows.")
    return rows

def churn_to_int(v):
    return 1 if str(v).strip().lower() == "yes" else 0

def safe_float(x):
    # TotalCharges can be blank for some rows
    if x is None:
        return None
    s = str(x).strip()
    if s == "":
        return None
    return float(s)

# ---- 2) Build 3NF SQLite (same "dimension + fact" concept as ocean_proximity) ----
def build_3nf_sqlite(db_path="telco.db"):
    print("=== BUILDING 3NF SQLITE DATA MODEL (TELCO CHURN) ===")

    print("\n[STEP 1] Loading CSV rows (NO pandas)…")
    rows = load_telco_rows()

    print("\n[STEP 2] Creating SQLite database and tables…")
    if os.path.exists(db_path):
        print("Existing DB found. Removing…")
        os.remove(db_path)

    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    print("Running SQL schema creation script…")
    cur.executescript(
        """
        DROP TABLE IF EXISTS customer_stats;
        DROP TABLE IF EXISTS customer;
        DROP TABLE IF EXISTS contract_dim;

        -- Dimension table (like ocean_proximity)
        CREATE TABLE contract_dim (
            contract_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE
        );

        -- Entity table (customer)
        CREATE TABLE customer (
            customer_id TEXT PRIMARY KEY,
            contract_id INTEGER NOT NULL,
            FOREIGN KEY (contract_id) REFERENCES contract_dim(contract_id)
        );

        -- Fact/stats table (features + target)
        CREATE TABLE customer_stats (
            customer_id TEXT PRIMARY KEY,
            tenure REAL,
            MonthlyCharges REAL,
            TotalCharges REAL,
            target INTEGER NOT NULL,
            FOREIGN KEY (customer_id) REFERENCES customer(customer_id)
        );
        """
    )
    print("Tables created.")

    print("\n[STEP 3] Building Contract dimension table…")
    contract_values = sorted({(r.get("Contract") or "").strip() for r in rows})
    if "" in contract_values:
        contract_values.remove("")

    cur.executemany(
        "INSERT OR IGNORE INTO contract_dim(name) VALUES (?)",
        [(v,) for v in contract_values],
    )
    conn.commit()

    cur.execute("SELECT contract_id, name FROM contract_dim")
    contract_map = {name: cid for (cid, name) in cur.fetchall()}
    print(f"Found {len(contract_map)} unique Contract values.")

    print("\n[STEP 4] Inserting customer + customer_stats…")
    customer_rows = []
    stats_rows = []

    for r in rows:
        cust_id = r["customerID"]
        contract_name = (r.get("Contract") or "").strip()
        contract_id = contract_map[contract_name]

        # target
        y = churn_to_int(r.get("Churn"))

        # numeric features
        tenure = safe_float(r.get("tenure"))
        monthly = safe_float(r.get("MonthlyCharges"))
        total = safe_float(r.get("TotalCharges"))

        customer_rows.append((cust_id, contract_id))
        stats_rows.append((cust_id, tenure, monthly, total, y))

    cur.executemany(
        "INSERT INTO customer (customer_id, contract_id) VALUES (?, ?)",
        customer_rows
    )
    cur.executemany(
        "INSERT INTO customer_stats (customer_id, tenure, MonthlyCharges, TotalCharges, target) VALUES (?, ?, ?, ?, ?)",
        stats_rows
    )

    conn.commit()
    conn.close()
    print("\n=== DONE! SQLite DB created at:", db_path, "===\n")

# Build it
build_3nf_sqlite("telco.db")


=== BUILDING 3NF SQLITE DATA MODEL (TELCO CHURN) ===

[STEP 1] Loading CSV rows (NO pandas)…
[1] Loading Telco CSV using csv.DictReader (NO pandas)…
[2] Loaded 7043 rows.

[STEP 2] Creating SQLite database and tables…
Running SQL schema creation script…
Tables created.

[STEP 3] Building Contract dimension table…
Found 3 unique Contract values.

[STEP 4] Inserting customer + customer_stats…

=== DONE! SQLite DB created at: telco.db ===

